In [1]:
import loompy
import os
import pandas as pd
import numpy as np
import re
import gc

/home/owenwhitley/anaconda3/envs/scvelo_conda/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9002. The TBB threading layer is disabled.
  warnings.warn(problem)


In [2]:
top_dir = "/home/owenwhitley/projects/su2c_v2"
data_dir = os.path.join(top_dir, 'data/preprocessed/scRNA/velocyto_GBM_GSCs_split_by_sample_no_G800_L')
merged_file = os.path.join(data_dir, 'GBM_GSCs_combined.loom')

In [3]:
merged_con = loompy.connect(merged_file)
merged_con.shape

(60675, 79862)

In [4]:
# Load DM coords
plot_data = pd.read_csv(os.path.join(data_dir, 'scvelo_plot_data_GSCs.csv'))
plot_data

,Unnamed: 0,nGene,nUMI,orig.ident,percent.mito,S.Score,G2M.Score,Phase,CC.Difference,Zhong_NPCs_upreg_AUC,...,Neftel_NPC2_AUC,Neftel_G1.S_AUC,Neftel_G2.M_AUC,Dev_IR_Diff,Stem_Astro_Diff,Dev_IR_Diff_zscore,Stem_Astro_Diff_zscore,DM1,DM2,unique_id
0,BT127_L_GTGAAGGGTCCGCTGA,1425,3339,BT127_L,0.216831,-0.026727,0.253973,G2M,-0.280700,0.104243,...,0.080527,0.045605,0.107647,0.023542,0.090076,1.128840,0.507793,0.001746,-0.012709,BT127_L_GTGAAGGGTCCGCTGA
1,BT127_L_GTCACAAAGGACAGAA,1852,4525,BT127_L,0.217680,0.266072,-0.072709,S,0.338782,0.112944,...,0.069466,0.094813,0.094662,-0.059629,0.089795,0.236915,0.501235,-0.000743,-0.024184,BT127_L_GTCACAAAGGACAGAA
2,BT127_L_GCTCTGTTCTTGCCGT,1743,5706,BT127_L,0.338416,0.026864,-0.107093,S,0.133957,0.084060,...,0.047279,0.082239,0.035772,-0.007674,0.131836,0.794075,1.482088,0.000854,-0.011782,BT127_L_GCTCTGTTCTTGCCGT
3,BT127_L_TTAGTTCTCACCTCGT,2617,7043,BT127_L,0.194377,0.001225,0.698205,G2M,-0.696980,0.236816,...,0.084929,0.012912,0.306327,-0.023635,0.175012,0.622919,2.489421,-0.000093,-0.019950,BT127_L_TTAGTTCTCACCTCGT
4,BT127_L_TTCGGTCTCCGCATAA,3166,10613,BT127_L,0.054650,-0.188679,-0.345842,G1,0.157163,0.071873,...,0.047171,0.000000,0.032355,-0.051998,0.055289,0.318754,-0.303820,0.002459,-0.004694,BT127_L_TTCGGTCTCCGCATAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,G946-K_L_TTTGGTTTCCTCCTAG,3598,13635,G946-K_L,0.070260,-0.182868,-0.347178,G1,0.164310,0.058077,...,0.033247,0.022221,0.037233,0.068235,0.009998,1.608117,-1.360508,0.007141,-0.000412,G946-K_L_TTTGGTTTCCTCCTAG
13996,G946-K_L_TTTCCTCAGCCACCTG,4694,21660,G946-K_L,0.093029,-0.158042,-0.272660,G1,0.114619,0.064947,...,0.071179,0.018317,0.029834,0.059739,-0.037694,1.517006,-2.473209,0.007971,0.001029,G946-K_L_TTTCCTCAGCCACCTG
13997,G946-K_L_TGTGGTAGTCATATCG,3397,13049,G946-K_L,0.092038,-0.113022,-0.223289,G1,0.110267,0.062871,...,0.050684,0.023347,0.018428,0.060646,-0.006145,1.526738,-1.737132,0.007513,0.000679,G946-K_L_TGTGGTAGTCATATCG
13998,G946-K_L_GACTACACAGTCTTCC,3883,15767,G946-K_L,0.072557,-0.191614,-0.265208,G1,0.073594,0.065343,...,0.026242,0.028902,0.040368,0.076105,-0.015896,1.692510,-1.964637,0.007357,0.000449,G946-K_L_GACTACACAGTCTTCC


In [5]:
plot_data['unique_id']

0         BT127_L_GTGAAGGGTCCGCTGA
1         BT127_L_GTCACAAAGGACAGAA
2         BT127_L_GCTCTGTTCTTGCCGT
3         BT127_L_TTAGTTCTCACCTCGT
4         BT127_L_TTCGGTCTCCGCATAA
                   ...            
13995    G946-K_L_TTTGGTTTCCTCCTAG
13996    G946-K_L_TTTCCTCAGCCACCTG
13997    G946-K_L_TGTGGTAGTCATATCG
13998    G946-K_L_GACTACACAGTCTTCC
13999    G946-K_L_CCTTCGAAGAAACGAG
Name: unique_id, Length: 14000, dtype: object

In [6]:
new_loom_ids = merged_con.ca['CellID']

In [7]:
# filter cells not in diffusion map
in_DM = np.isin(new_loom_ids, plot_data['unique_id'].to_numpy())

In [8]:
sum(in_DM)

14000

In [9]:
len(plot_data)

14000

In [10]:
plot_data = plot_data.rename(index = plot_data['unique_id'])

In [11]:
# create subsetted loom. note that subsetting through loompy API is way too slow.
# Just load whole matrix into memory and subset
gc.collect()
subs_spl = merged_con.layers['spliced'][:, :]
subs_spl = subs_spl[:, in_DM]
gc.collect()
subs_uns = merged_con.layers['unspliced'][:, :]
subs_uns = subs_uns[:, in_DM]
gc.collect()

0

In [12]:
subs_ca = {}
subs_ca['CellID'] = merged_con.ca['CellID'][np.where(in_DM)]
for k in plot_data.columns:
    subs_ca[k] = plot_data[k].loc[subs_ca['CellID']].to_numpy()
# subs_ca.keys()

In [13]:
subs_file = os.path.join(data_dir, 'GSCs_subs.loom')
if os.path.exists(subs_file):
    os.remove(subs_file)
loompy.create(subs_file, {'' : subs_spl, 'spliced' : subs_spl, 'unspliced' : subs_uns}, merged_con.ra, subs_ca)